In [24]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp

### 1. Link prediction with GNN
Assume we are given a graph g with incomplete data, for example, only 50% of the edges are present. 

The goal is to predict **whether there is an edge** between any 2 nodes in g.

In [28]:
# Load graph
import dgl.data
dataset=dgl.data.CoraGraphDataset()
g=dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


#### (a) Prepare graph
The graph we work **g_main** on is formed from the original graph by taking out 20% of its edges (test edges). 
1. **Training**: train g_main on known positive edges and negative edges
2. **Testing**: test g_main on uknown positive edges and negative edges


In [31]:
# randomly permute edge ids
edge_ids=np.arange(g.num_edges())
edge_ids=np.random.permutation(edge_ids)

train_size=int(0.8*g.num_edges())
train_mask=edge_ids[:train_size]
test_mask=edge_ids[train_size:]

# g_main= original g with all test edges removed
g_main=dgl.remove_edges(g,test_mask)

#### (b) Obtain positive and negative samples
1. Existing edges in graph = positive examples
2. Non-existing edges (node pairs with no edges) = negative examples.
3. Form train set and test set by the positive+negative examples
4. Evaluate the model with AUC metric

In [32]:
# extract edges
u,v=g.edges() # u=source nodes, v= destination nodes

# (1) Get positive edges for training and testing of g_main
train_pos_u, train_pos_v=u[train_mask], v[train_mask]
test_pos_u, test_pos_v=u[test_mask], v[test_mask]

# (2) Get negative edges for training and testing of g_main
# create sparse adj matrix
adj=sp.coo_matrix((np.ones(len(u)), (u.numpy(),v.numpy())))
adj=adj.todense()+np.eye(g.num_nodes())

u_neg, v_neg=np.where(adj==0)

# sample g.num_edges() negative edges
neg_ids=np.random.choice(len(u_neg),g.num_edges())
train_neg_u,train_neg_v=u_neg[neg_ids[:train_size]], v_neg[neg_ids[:train_size]]

test_neg_u,test_neg_v=u_neg[neg_ids[train_size:]], v_neg[neg_ids[train_size:]]

print(f"Train edges: {len(train_pos_u)+len(train_neg_v)} | positive edges: {len(train_pos_u)} | negative edges: {len(train_neg_u)}")
print(f"Test edges: {len(test_pos_u)+len(test_neg_v)} | positive edges: {len(test_pos_u)} | negative edges: {len(test_neg_u)}")



Train edges: 16888 | positive edges: 8444 | negative edges: 8444
Test edges: 4224 | positive edges: 2112 | negative edges: 2112


In [33]:
# create 4 types of graphs for computing scores in training and testing

# graphs used in computing scores in training
# (1) positive train graph -> contain all positive edges
g_train_pos=dgl.graph((train_pos_u,train_pos_v),num_nodes=g.num_nodes())

# (2) negative train graph -> contain all positive edges
g_train_neg=dgl.graph((train_neg_u,train_neg_v),num_nodes=g.num_nodes())



# graphs used in computing scores in testing
# (3) positive test graph -> contain all positive edges
g_test_pos=dgl.graph((test_pos_u,test_pos_v),num_nodes=g.num_nodes())
# (4) negative test graph -> contain all positive edges
g_test_neg=dgl.graph((test_neg_u,test_neg_v),num_nodes=g.num_nodes())


### 2. GNN with SageConv
dgl.nn.SAGEConv(in_dim, out_dim) updates in the following way

\begin{align*}
h_i^{(l+1)}&= W.\text{concat}(h_i^{(l)},h_{N(i)}^{(l+1)})+b \ \text{with} \\ h_{N(i)}^{(l+1)}&=\text{Mean}\{h_j^{(l)}, j\in N(i)\} 
\end{align*}

Here is our **model structure**
<center>
input -> SAGEConv1 -> relu -> SAGEConv2 -> predictor
<end><center>


In [39]:
import dgl.function as fn
from dgl.nn import SAGEConv

from sklearn.metrics import roc_auc_score # for computing auc metric

class GraphSage(nn.Module):
    def __init__(self,in_dim,hidden_dim):
        super().__init__()
        self.conv1=SAGEConv(in_dim,hidden_dim,aggregator_type="mean")
        self.conv2=SAGEConv(hidden_dim,hidden_dim,aggregator_type="mean")
    
    def forward(self,g,features):
        # g=input graph, features= input node features
        h=self.conv1(g,features)
        h=F.relu(h)
        h=self.conv2(g,h)
        return h
    
    def predict(self,g,h):
        # compute dot product of src & dst features
        #         store it as a new feature for g.edata
        with g.local_scope():
            g.ndata['h']=h
            # create a new edge attribute 'score' = src_dot_dst
            g.apply_edges(fn.u_dot_v('h','h','score'))
            return g.edata['score'][:,0]
        
    def loss(self,pos_scores,neg_scores):
        # pos_scores = scores for positive edges
        # neg_scores = scores for negative edges
        scores=torch.cat([pos_scores,neg_scores])
        labels=torch.cat([torch.ones(pos_scores.shape[0]),torch.zeros(neg_scores.shape[0])])

        return F.binary_cross_entropy_with_logits(scores,labels)
    
    def auc_score(self,pos_scores,neg_scores):
        # roc_auc_score only accepts numpy array as inputs
        scores=torch.cat([pos_scores,neg_scores]).detach().numpy()
        labels=torch.cat([torch.ones(pos_scores.shape[0]),
                          torch.zeros(neg_scores.shape[0])]).detach().numpy()
        return roc_auc_score(labels,scores)



### 3. Train and Test

In [40]:
# train loop: train graph g_main with information from g_train_pos and g_train_neg

# train one epoch
def train(model,g_main,g_train_pos,g_train_neg,optimizer):
    model.train()
    
    # forward and backward
    optimizer.zero_grad()

    # prediction on g_main
    h=model(g_main,g_main.ndata['feat']) # [num_nodes,feat_dim]

    # prediction scores 
    pos_scores=model.predict(g_train_pos,h)
    neg_scores=model.predict(g_train_neg,h)
    loss=model.loss(pos_scores,neg_scores)
    loss.backward()
    optimizer.step()

    # compute auc
    auc_score=model.auc_score(pos_scores,neg_scores)

    return loss, auc_score
    


In [41]:
@torch.no_grad()
def evaluate(model,g_main,g_test_pos, g_test_neg):
    model.eval()
    h=g_main.ndata['feat'] # these features are extracted after training is finished
    
    # forward
    pos_scores=model.predict(g_test_pos,h)
    neg_scores=model.predict(g_test_neg,h)

    loss=model.loss(pos_scores,neg_scores)

    auc_score=model.auc_score(pos_scores,neg_scores)

    return loss,auc_score

In [42]:
# hyperparameters
in_dim=g_main.ndata['feat'].shape[-1]
hidden_dim=16
num_epochs=100

model=GraphSage(in_dim,hidden_dim)
print(f"{sum(p.numel() for p in model.parameters())/1e6} million parameters")

0.0464 million parameters


In [43]:
optimizer=torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(num_epochs):
    train_loss, train_auc=train(model,g_main,g_train_pos, g_train_neg,optimizer)
    if epoch%5==0 or epoch==num_epochs-1:
        print(f"Epoch: {epoch} | train_loss: {train_loss:.4f} |  train_auc: {train_auc:.4f}  ")

Epoch: 0 | train_loss: 0.7260 |  train_auc: 0.5282  
Epoch: 5 | train_loss: 0.6937 |  train_auc: 0.6533  
Epoch: 10 | train_loss: 0.6889 |  train_auc: 0.6555  
Epoch: 15 | train_loss: 0.6792 |  train_auc: 0.7847  
Epoch: 20 | train_loss: 0.6559 |  train_auc: 0.7868  
Epoch: 25 | train_loss: 0.6134 |  train_auc: 0.8098  
Epoch: 30 | train_loss: 0.5766 |  train_auc: 0.8076  
Epoch: 35 | train_loss: 0.5592 |  train_auc: 0.8183  
Epoch: 40 | train_loss: 0.5414 |  train_auc: 0.8426  
Epoch: 45 | train_loss: 0.5179 |  train_auc: 0.8680  
Epoch: 50 | train_loss: 0.4943 |  train_auc: 0.8936  
Epoch: 55 | train_loss: 0.4768 |  train_auc: 0.9097  
Epoch: 60 | train_loss: 0.4603 |  train_auc: 0.9233  
Epoch: 65 | train_loss: 0.4414 |  train_auc: 0.9350  
Epoch: 70 | train_loss: 0.4243 |  train_auc: 0.9445  
Epoch: 75 | train_loss: 0.4080 |  train_auc: 0.9520  
Epoch: 80 | train_loss: 0.3927 |  train_auc: 0.9575  
Epoch: 85 | train_loss: 0.3753 |  train_auc: 0.9630  
Epoch: 90 | train_loss: 0.3577

In [44]:
# evaluate on test graph
with torch.no_grad():
    test_loss, test_auc=evaluate(model,g_main,g_test_pos, g_test_neg)
    print(f"test_loss: {test_loss:.4f} | test_auc: {test_auc:.4f}")

test_loss: 0.6913 | test_auc: 0.7948
